HLM 및 MLM은 간 및 마우스의 간 대사효소와 화합물을 30분 동안 반응시킨 후, 
<대사되지 않고 남아있는 화합물의 양>을(%) LC-MS/MS로 측정함으로써 화합물의 간 대사효소에 대한 안정성을 평가한 데이터
LC-MS/MS : 액체 크로마토그래피 질량 분석(액체 크로마토그래피의 물리적 분리 기능과 질량 분석 기능을 결합한 분석 화학 기술)
복잡한 샘플에서 특정 화합물을 분리, 정량 및 식별하는데 널리 사용

대사 안정성이 낮은 물질의 경우 체내 소실이 빨리, 많이 된다.
적절히 체내에 남아 있으면서 약효가 발휘되어야한다.
가능한 한 신약개발 초기부터 대사안정성이 높은 물질을 선별해야한다.

신약개발의 metabolity 최적화 -> 실험적인 방법과 컴퓨터를 활용한 전략을 병용한 사례가 있음.
일부 약물의 경우 clearance가 높았다고...

------------------------------------------------------------------------------------------------

간 조직을 분리해서 얻은 hepatocyte가 metabolism을 평가하는데 가장 좋지만,
S9, Microsome, cytosol도 있음
간 조직을 경질화 한다음, 낮은 speed로 원심분리하여 얻은 것이 s9,
그 다음 빠른 속도로 원심분리하여, 위에 뜬게 cytosol, 아래 가라앉은게 microsomal
가장 대표적인 enzyme을 포함한 microsome을 이용한 대사 평가가 가장 많이 사용
liver Microsomes in Buffer(Pb 7.4) + Drug -> (NDAPH, 37도에서, CYP450) -> Drug + Drug -OH

주최측에서 준 것은 Single point assay(with / without NADPH가 가능)

대사 안정성 분류 기준
반감기 기준으로 할 경우, 30분 기준 <50% 이면 안정한 물질, 중간은 moderate, <12.4%면 불안정한 물질.
clearance 기준으로 할 경우, ~
다만 우리가 가진건 단지 30분 후 대사물의 양일뿐

------------------------------------------------------------------------------------------------

column들의 의미

id: 화합물의 아이디.
SMILES: 화합물의 SMILES (Simplified Molecular Input Line Entry System) 표기법.
MLM: 화합물의 MLM (Microsomal Liver Metabolism) 값.
HLM: 화합물의 HLM (Human Liver Microsomes) 값.
AlogP: 화합물의 AlogP 값.
Molecular_Weight: 화합물의 분자량.
Num_H_Acceptors: 화합물의 수소 수용체 개수.
Num_H_Donors: 화합물의 수소공여체 개수.
Num_RotatableBonds: 화합물의 회전 가능 결합 개수.
LogD: 화합물의 LogD 값.
Molecular_PolarSurfaceArea: 화합물의 극성 표면적.

------------------------------------------------------------------------------------------------

Lipinski's Rule

- **Rule of five(RO5)** 라고도 불리며 Druglikeness를 판단하는데 사용
- 1997년 Pfizer의 과학자 Christopher A. Lipinski는 구강복용하는 약물이 상대적으로 크기가 작고 지용성이라는 것을 관찰하여 이 규칙을 고안

- ADME(pharmacokinetic)에 기반한 프로파일
- Molecular weight < 500 Dalton
- Octanol-water partition coefficient (LogP) < 5
- Hydrogen bond donors < 5
- Hydrogen bond acceptors < 10

------------------------------------------------------------------------------------------------

In [1]:
import sys
import pandas as pd
import numpy as np
import os
import operator
import string
import re
import random
import warnings
warnings.filterwarnings('ignore')
import platform
import json, pickle
import networkx as nx

from copy import deepcopy
from math import sqrt
from random import shuffle
from collections import OrderedDict
from scipy import stats
from IPython.display import SVG

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU

from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.Chem import AllChem, rdDepictor, MolFromSmiles, Descriptors, rdMolDescriptors, Crippen, QED, EState, Lipinski, MolSurf, Fragments

import deepchem as dc
from deepchem.feat.molecule_featurizers import MolGraphConvFeaturizer
from deepchem.feat import RDKitDescriptors

import torch_geometric
from torch.nn import Dropout, BatchNorm1d
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_add_pool, global_max_pool

import sklearn as sk
from sklearn.utils import check_random_state
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

import deepchem as dc
from deepchem.splits.splitters import ScaffoldSplitter

# 시각화 라이브러리
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
# What version of Python do you have?
has_gpu = torch.cuda.is_available()
# has_mps = torch.backends.mps.is_built()
device = "cuda" if has_gpu else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if has_gpu else "NOT AVAILABLE")
print(f"Target device is {device}")

Python Platform: macOS-13.5.2-arm64-arm-64bit
PyTorch Version: 2.0.1

Python 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:41:52) [Clang 15.0.7 ]
Pandas 2.1.0
Scikit-Learn 1.3.0
GPU is NOT AVAILABLE
Target device is cpu


In [3]:
CFG = {
    'SEED': 42,
    'TRAIN_BATCH_SIZE' : 32,
    'TEST_BATCH_SIZE' : 32,
    'LR' : 0.0002,
    'LOG_INTERVAL' : 20,
    'NUM_EPOCHS' : 50,
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    check_random_state(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')
train.head()

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43


In [25]:
def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

In [26]:
def atom_features(atom, mol): # 52
    features = one_of_k_encoding(atom.GetSymbol(),['N', 'S', 'F', 'Cl', 'Se', 'Br', 'O', 'C', 'I', 'P'])
    features += one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    features += one_of_k_encoding(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    features += one_of_k_encoding(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    features.append(atom.GetIsAromatic())
    features += one_of_k_encoding(atom.GetFormalCharge(), [-1, 0, 1])  # 형식 전하
    features += one_of_k_encoding(str(atom.GetHybridization()), ['SP', 'SP2', 'SP3', 'SP3D', 'SP3D2'])  # 혼성
    
    return np.array(features)

In [27]:
# Returns : 원자 개수, 원자 특성 행렬, 인접 행렬
def smile_to_graph(smile):
    # SMILES 문자열로부터 분자 그래프 데이터 생성
    mol = Chem.MolFromSmiles(smile)
    
    # mol.GetNumAtoms() : 분자에 소속되어 있는 원자의 개수
    c_size = mol.GetNumAtoms()
    
    features = []

    # 분자에 소속되어 있는 원자들을 순회하면서 원자 특성 정보 수집
    for atom in mol.GetAtoms():
        feature = atom_features(atom, mol)
        # 정규화?
        features.append( feature / sum(feature) )

    edges = []
    # 분자를 이루는 원자들의 연결 구조 정보를 순회하면서 인접 정보 수집
    # 연결 구조 정보 : 시작 원자 index, 끝 원자 index
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])

    # 연결 구조 정보를 통한 방향 그래프 생성(양방향)
    g = nx.Graph(edges).to_directed()
    edge_index = []
    for e1, e2 in g.edges:
        edge_index.append([e1, e2])

    return c_size, features, edge_index

In [28]:
smile_graph = {}
for df in [train, test]:
    smiles = df['SMILES']
    for smile in smiles:
        g = smile_to_graph(smile)
        smile_graph[smile] = g

In [29]:
dataset = dc.data.NumpyDataset(X=np.array(train['SMILES']), ids=train['SMILES'].values)
splitter = dc.splits.ScaffoldSplitter()
train_dataset, valid_dataset = splitter.train_test_split(dataset, frac_train=0.85, seed=CFG['SEED'])
train_df = train[train['SMILES'].isin(train_dataset.ids)]
valid_df = train[train['SMILES'].isin(valid_dataset.ids)]

In [ ]:
class TestbedDataset(Dataset):
    def __init__(self, df, mode='train', transform=None, pre_transform=None, smile_graph=None, scaler=None):
        super(TestbedDataset, self).__init__(root=None, transform=transform, pre_transform=pre_transform)
        self.mode = mode
        self.smile_graph = smile_graph
        self.scaler = scaler
        
        self.xd = df['SMILES'].tolist()

        if self.mode == 'train':
            self.y = df[['MLM', 'HLM']].values.tolist()
        else:  # 'test' mode
            self.y = [None] * len(self.xd)

        if self.mode == 'train':
            exclude_cols = ['SMILES', 'MLM', 'HLM']
            additional_data = df.drop(exclude_cols, axis=1)
        
        else:
            additional_data = df.drop('SMILES', axis=1)

        if self.mode == 'train':
            self.scaler = StandardScaler()
            self.scaled_data = self.scaler.fit_transform(additional_data)
        elif self.mode == 'test' and self.scaler is not None:
            self.scaled_data = self.scaler.transform(additional_data)
        else:
            raise ValueError("Scaler should be provided for test mode.")

    def len(self):
        return len(self.xd)

    def get(self, idx):
        smiles = self.xd[idx]
        labels = self.y[idx]
        additional_features = self.scaled_data[idx]

        c_size, features, edge_index = self.smile_graph[smiles]

        data = Data(x=torch.Tensor(features),
                    edge_index=torch.LongTensor(edge_index).transpose(1, 0))
        
        if labels:
            data.y = torch.FloatTensor([labels])
        data.c_size = torch.LongTensor([c_size])
        data.additional_features = torch.Tensor(additional_features)

        return data

In [ ]:
train_dataset = TestbedDataset(df=train_df, mode='train', smile_graph=smile_graph)
val_dataset = TestbedDataset(df=valid_df, mode='train', smile_graph=smile_graph, scaler=train_dataset.scaler)
test_dataset = TestbedDataset(df=test, mode='test', smile_graph=smile_graph, scaler=train_dataset.scaler)

In [ ]:
batch = next(iter(train_dataset))
batch

In [ ]:
class GATNet(torch.nn.Module):
    def __init__(self, n_output=2, num_features_xd=52, output_dim=128, dropout=0.2, num_additional_features=66):
        super(GATNet, self).__init__()

        dim = 32
        heads = 4  # Number of attention heads

        # 약물 분자 표현을 위한 GAT 층 구성
        self.dropout = Dropout(dropout)
        self.relu = ReLU()
        self.n_output = n_output

        self.conv1 = GATConv(num_features_xd, dim, heads=heads, concat=True)
        self.bn1 = BatchNorm1d(dim * heads)
        self.conv2 = GATConv(dim * heads, dim, heads=heads, concat=True)
        self.conv3 = GATConv(dim * heads, dim, heads=heads, concat=True)
        self.fc1_xd = Linear(dim * heads, output_dim)

        # Additional Features Fully Connected Layer
        self.fc_additional = Linear(num_additional_features, output_dim)

        # Concatenation
        self.fc1 = Linear(2 * output_dim, 1024)
        self.fc2 = Linear(1024, 256)
        self.out = Linear(256, self.n_output)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        additional_features = data.additional_features.view(-1, 66)

        # Molecular features
        x = F.elu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = F.elu(self.conv2(x, edge_index))
        x = F.elu(self.conv3(x, edge_index))
        x = global_add_pool(x, batch)
        x = F.elu(self.fc1_xd(x))
        x = self.dropout(x)

        # Process additional features
        additional_out = self.fc_additional(additional_features)
        additional_out = F.elu(additional_out)
        additional_out = self.dropout(additional_out)

        # Concatenation
        xc = torch.cat((x, additional_out), 1)
        xc = self.fc1(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        return out

In [ ]:
def custom_loss(y_pred, y_true):
    loss1 = nn.mse_loss(y_pred[:, 0].flatten(), y_true[:, 0].flatten())
    loss2 = nn.mse_loss(y_pred[:, 1].flatten(), y_true[:, 1].flatten())
    return (loss1**0.5 + loss2**0.5) / 2

In [ ]:
def training(model, device, train_loader, optimizer, epoch):
    print('Training on {} samples...'.format(len(train_loader.dataset)))
    model.train()
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)

        # Adjusting the shape of target data to match the output
        loss1 = F.mse_loss(output[:, 0].flatten(), data.y[:, 0].to(device).flatten())
        loss2 = F.mse_loss(output[:, 1].flatten(), data.y[:, 1].to(device).flatten())
        loss = (loss1**0.5 + loss2**0.5) / 2

        loss.backward()
        optimizer.step()

        if batch_idx % LOG_INTERVAL == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                                                           batch_idx * train_loader.batch_size,
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.item()))

In [ ]:
def predicting(model, device, loader, mode='validation'):
    model.eval()

    # 학습을 통해 예측된 값을 저장합니다.
    total_preds = []

    # 실제 값을 저장합니다.
    total_labels = []

    print('Make prediction for {} samples...'.format(len(loader.dataset)))
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            total_preds.append(output.cpu().numpy())
            
            # 만약 mode가 'test'가 아니라면 실제 값을 저장
            if mode != 'test':
                total_labels.append(data.y.cpu().numpy())
    
    # 리스트를 numpy 배열로 변환합니다.
    if mode == 'test':
        return np.vstack(total_preds)
    else:
        return np.vstack(total_labels), np.vstack(total_preds)

In [ ]:
TRAIN_BATCH_SIZE = CFG['TRAIN_BATCH_SIZE']
TEST_BATCH_SIZE = CFG['TEST_BATCH_SIZE']
LR = CFG['LR']
LOG_INTERVAL = CFG['LOG_INTERVAL']
NUM_EPOCHS = CFG['NUM_EPOCHS']

In [ ]:
def rmse(y,f):
    rmse = np.sqrt(((y - f)**2).mean(axis=0))
    return rmse

In [ ]:

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=TRAIN_BATCH_SIZE)

model = GATNet().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

model_file_name = 'GATNet.model'

for epoch in range(NUM_EPOCHS):
    training(model, device, train_loader, optimizer, epoch+1)
    G, P = predicting(model, device, val_loader)
    ret = rmse(G,P)

    print('rmse at epoch {}; best_rmse: {:.4f}'.format(epoch, ret))

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False)

In [ ]:
preds = predicting(model, device, test_loader, mode='test')
preds

In [ ]:
sample_submission = pd.read_csv('Data/sample_submission.csv')
sample_submission

In [ ]:
sample_submission['MLM'] = preds[:, 0]
sample_submission['HLM'] = preds[:, 1]
sample_submission

In [ ]:
sample_submission.to_csv('submission.csv', index=False)